In [4]:
import pandas as pd
import os 

In [58]:
os.chdir('/content/drive/MyDrive/OID/csv_folder')
classes_data= pd.read_csv('class-descriptions-boxable.csv',header=None )
classes_data.head()

,0,1
0,/m/011k07,Tortoise
1,/m/011q46kg,Container
2,/m/012074,Magpie
3,/m/0120dh,Sea turtle
4,/m/01226z,Football


In [48]:
classes = ['Apple', 'Orange', 'Tomato']

class_string = []

for i in classes:
  req_class = classes_data.loc[classes_data[1]==i]
  string = req_class.iloc[0][0]
  print(string)
  class_string.append(string)

print(class_string)

/m/014j1m
/m/0cyhj_
/m/07j87
['/m/014j1m', '/m/0cyhj_', '/m/07j87']


In [49]:
annotation_data = pd.read_csv('train-annotations-bbox.csv',
                              usecols=['ImageID',
                              'LabelName',
                              'XMin',
                              'XMax',
                              'YMin',
                              'YMax'])
annotation_data.head()

,ImageID,LabelName,XMin,XMax,YMin,YMax
0,000002b66c9c498e,/m/01g317,0.012500,0.195312,0.148438,0.587500
1,000002b66c9c498e,/m/01g317,0.025000,0.276563,0.714063,0.948438
2,000002b66c9c498e,/m/01g317,0.151562,0.310937,0.198437,0.590625
3,000002b66c9c498e,/m/01g317,0.256250,0.429688,0.651563,0.925000
4,000002b66c9c498e,/m/01g317,0.257812,0.346875,0.235938,0.385938


In [50]:
filtered_class_data = annotation_data.loc[annotation_data['LabelName'].isin(class_string)].copy()
filtered_class_data.head()

,ImageID,LabelName,XMin,XMax,YMin,YMax
3438,00020ebf74c4881c,/m/0cyhj_,0.786250,0.985625,0.466853,0.706816
8735,0006124f7e35107e,/m/07j87,0.061758,0.793349,0.035642,0.560861
8736,0006124f7e35107e,/m/07j87,0.235946,0.965162,0.502354,0.944855
16565,000d1976fc8ebfe7,/m/0cyhj_,0.000000,0.030625,0.486308,0.565628
16566,000d1976fc8ebfe7,/m/0cyhj_,0.000000,0.053750,0.730878,0.834750


In [51]:
filtered_class_data['classNumber']=''
filtered_class_data['center x']=''
filtered_class_data['center y']=''
filtered_class_data['width']=''
filtered_class_data['height']=''

In [52]:
for i in range(len(class_string)):
  filtered_class_data.loc[filtered_class_data['LabelName']==class_string[i] ,'classNumber'] = i

filtered_class_data['center x']=(filtered_class_data['XMax']+filtered_class_data['XMin'])/2
filtered_class_data['center y']=(filtered_class_data['YMax']+filtered_class_data['YMin'])/2

filtered_class_data['width'] = filtered_class_data['XMax']-filtered_class_data['XMin']
filtered_class_data['height'] = filtered_class_data['YMax']-filtered_class_data['YMin']

Yolo_values= filtered_class_data.loc[:,['ImageID','classNumber','center x','center y','width','height']].copy()
Yolo_values.head()



,ImageID,classNumber,center x,center y,width,height
3438,00020ebf74c4881c,1,0.885938,0.586835,0.199375,0.239963
8735,0006124f7e35107e,2,0.427553,0.298252,0.731591,0.525219
8736,0006124f7e35107e,2,0.600554,0.723604,0.729216,0.442501
16565,000d1976fc8ebfe7,1,0.015312,0.525968,0.030625,0.079320
16566,000d1976fc8ebfe7,1,0.026875,0.782814,0.053750,0.103872


In [55]:
save_path3='/content/drive/MyDrive/OID/Dataset/labels/train'

In [56]:
image_path= '/content/drive/MyDrive/OID/Dataset/images/train'

os.chdir(image_path)

In [54]:
save_path3='/content/drive/MyDrive/OID/Dataset/labels/train'

In [57]:
from numpy.lib.type_check import imag
for current_dir,dirs,files in os.walk('.'):
  for f in files:
    if f.endswith('.jpg'):
      image_title = f[:-4]

      Yolo_file =Yolo_values.loc[Yolo_values['ImageID']==image_title]
      df=Yolo_file.loc[:,['classNumber','center x','center y','width','height']].copy()
      save_path= save_path3+'/'+image_title+'.txt'
      df.to_csv(save_path, header=False, index=False, sep=' ')